## Instalar e importar módulos

In [1]:
#Beautiful Soup de bs4 y requests sirven para WebScraping (ya instalados con Anaconda)
#Selenium sirve para automatizar el navegador(desde Anaconda Prompt ejecutar: "pip install selenium")
#PyAutoGUI sirve para automatizar movimientos de mouse, clicks, teclado ("pip install pyautogui")
#pyperclip para trabajar con portapapeles ("pip install pyperclip")
#pandas para armar el dataframe, time para hacer pausas entre comandos

In [206]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
import pandas as pd
import pyautogui as pg
import pyperclip as pc
import time

In [3]:
#Parsers de Scraping: lxml y html5lib (ya instalados con Anaconda)
#Webdriver de Chrome: Ir a "https://sites.google.com/a/chromium.org/chromedriver/downloads" para descargar un WebDriver
#para usarlo con Selenium  
#Revisar que sea la misma versión, el mío es ChromeDriver 83.0.4103.39 para Windows, para revisar la versión correcta
#ir a las opciones a la esquina superior derecha de Chrome > Help > About Google Chrome

## Entrar a PowerCampus > Clases Virtuales con Selenium

In [207]:
browser = webdriver.Chrome("C:\Program Files (x86)\chromedriver83.exe") #Acá puse el driver
browser.get("https://autoservicio2.up.edu.pe/ss/Home.aspx")
barra_user = browser.find_element_by_id("ctl00_ucUserLogin_lvLoginUser_ucLoginUser_lcLoginUser_UserName")
barra_password = browser.find_element_by_id("ctl00_ucUserLogin_lvLoginUser_ucLoginUser_lcLoginUser_Password")
barra_user.send_keys("jm.huamanpi") #Acá poner su usuario UP
barra_password.send_keys("831134DD") #Acá la contraseña
barra_password.send_keys(Keys.ENTER)
boton_clases = browser.find_element_by_link_text("Clases")
boton_clases.send_keys(Keys.ENTER)
boton_clasesvirtuales = browser.find_element_by_link_text("Clases Virtuales")
boton_clasesvirtuales.send_keys(Keys.ENTER)
pagina_base = browser.current_url
pagina_base

'https://autoservicio2.up.edu.pe/ss/UP/AdminClasesVirtuales/Forms/ListarClasesVirtuales.aspx'

## Navegar cursos y obtener links con Beautiful Soup o requests

### Videos de un curso

In [208]:
boton_desplegable = browser.find_element_by_id("select2-searchCursos-container")
boton_desplegable.click()
busqueda = browser.find_element_by_class_name("select2-search__field")
busqueda_desplegable = browser.find_element_by_xpath('//*[@id="ctl00_Html1"]/body/span/span/span[1]/input')
busqueda_desplegable.send_keys("administracion") #Nombre del curso
busqueda_desplegable.send_keys(Keys.ENTER)

In [229]:
# #Scraping
# pedido = requests.get(pagina_curso) #pedido GET request
# pedido
# html = pedido.content #su contenido
# soup = BeautifulSoup(html, "lxml") #soup es el objeto, lmxl es el parser
# #Ahora lo guardo en un archivo 'pedido.html'
# with open('pedido.html', "wb") as file:
#     file.write(soup.prettify("utf-8"))
# #Este pedido cambia con el Javascript así que no sirve, por eso lo sacaré con PyAutoGUI

In [209]:
#Scraping
time.sleep(1.8)
pg.hotkey('ctrl','shift','i')
time.sleep(1.5)
for i in range(0,2):
    pg.press('up')
pg.hotkey('shift','f10')
time.sleep(1)
for i in range(0,5):
    pg.press('down')
pg.press('right')
for i in range(0,2):
    pg.press('down')
pg.press('enter')
html = pc.paste() 
html
soup = BeautifulSoup(html, "lxml") #soup es el objeto, lmxl es el parser
#Ahora lo guardo en un archivo 'html.html'
with open('html.html', "wb") as file:
    file.write(soup.prettify("utf-8"))
#time.sleep(1)
#pg.click(pg.locateCenterOnScreen('HTMLtag.png'), button='right')
#time.sleep(1)
#pg.click(pg.locateCenterOnScreen('Copy.png'))
#time.sleep(1)
#pg.click(pg.locateCenterOnScreen('Copyouterhtml.png'))

## Obtener nombres de cursos

In [228]:
type(soup)
cursoslista = soup.find('select', {"id":"searchCursos"})
cursostexto = cursoslista.contents
cursos=[]
for i in range(1,len(cursostexto)):
    cursos.append(cursostexto[i].text)
cursos[:5]

['Acto Jurídico(A)',
 'Administración de Operaciones(A)',
 'Administración de Operaciones(B)',
 'Administración de Operaciones(C)',
 'Administración de Operaciones(D)']

## Obtener cabeceras y datos

### Cabeceras

In [263]:
tabla = soup.find('table', {"class":"table","id":"tablaResultado"})
cabeceraslista = tabla.thead.tr
cabecerastexto = cabeceraslista.contents
cabeceras=[]
for i in range(1,len(cabecerastexto)-1,2):
    cabeceras.append(cabecerastexto[i].text)
cabeceras

['Curso', 'Sección', 'Fecha', 'Nombre de la sesión', 'Duración', 'Link']

### Datos

In [275]:
datos = tabla.tbody
claseslista = datos.contents
claseslista[1].find_all('td')

[<td>Administración de Operaciones</td>,
 <td>A</td>,
 <td>2020-04-02</td>,
 <td>Sesión Jueves 02 Abril 2020</td>,
 <td>1:44:06</td>,
 <td> <a href="https://ca.bbcollab.com/recording/50a74f22db544b69842a0db581be3746" target="_blank">ingresar</a></td>]

###  Obtener urls ca.bbcollab

In [93]:
enlaces = browser.find_elements_by_xpath('.//a[text()="ingresar"]')
urls = [] #Acá están los urls
for n in enlaces:
    urls.append(n.get_attribute('href'))
urls
len(urls)

36

### Obtener urls cloudfront

In [95]:
urls2 = [] #Acá estarán los nuevos urls
for a in range(0,len(urls)):
    browser.get(urls[a])
    if a == 0:
        pg.hotkey('ctrl','shift','i') #Abrir las herramientas del desarrollador
        pg.hotkey('winleft','up') #Maximizar la ventana
        time.sleep(1.5)
        pg.click(pg.locateCenterOnScreen('Network.png'))
    time.sleep(8)
    pg.click(pg.locateCenterOnScreen('Media.png'),button='right')
    pg.click(pg.locateCenterOnScreen('Open.png'))
    browser.switch_to.window(browser.window_handles[1])
    urls2.append(browser.current_url)
    pg.hotkey('ctrl','w')
    browser.switch_to.window(browser.window_handles[0])
urls2
len(urls2)

1

## Organizar tabla con pandas y exportar como csv